# Imports

In [1]:
import pandas as pd
import time
from string_grouper import match_strings
from neo4j import GraphDatabase
from helper.dataset_splitter import DataSetSplitter

ModuleNotFoundError: No module named 'string_grouper'

### Um die aktuellen Daten aus MISP zu erhalten, kann die nachfolgende Codezeile ausgeführt werden. Per default liegt bereits eine exportierte MISP csv-Datei bereit.


In [ ]:
# !python3 ../helper/get_csv.py -f ../data/misp_events.csv

In [ ]:
data_set_splitter = DataSetSplitter()
data_set_splitter.split_datasets("../data")

In [ ]:
pd.set_option('display.max_colwidth', None)
domains = pd.read_csv('../data/splitted_datasets/export_domain.csv', usecols=['uuid','event_id', 'value'])

In [ ]:
domains

In [ ]:
print('The shape: %d x %d' % domains.shape)
print(domains.head())

# String matching with string_grouper

In [ ]:
matches = match_strings(domains['value'], master_id=domains['event_id'])
print(matches)

### drop combinations with similar events 

In [ ]:
diff_events_only = matches[matches.left_event_id != matches.right_event_id]
print(diff_events_only)

# Chose similarity level

In [ ]:
mask = diff_events_only["similarity"] < 2.0
df = diff_events_only[mask]
df

In [ ]:
df = df.drop(columns = ["left_index", "right_index"])
df.head(30)

# Preperation: Create a list of unique event_id's

In [ ]:
temp_df = df['left_event_id'].append(df['right_event_id'])
unique_event_nodes = temp_df.drop_duplicates()
unique_event_nodes
unique_event_nodes_list = unique_event_nodes.to_list()
unique_event_nodes_list
print(unique_event_nodes_list)

In [ ]:
len(unique_event_nodes_list)

# Create for every unique event_id one node

In [ ]:
unique_event_nodes_execution_commands = []

for i in unique_event_nodes_list:
   event_node = "(e:Event {event_id: "+ "'" + str(i) + "'" + "})"
   neo4j_create_statemenet = "CREATE" + event_node
   unique_event_nodes_execution_commands.append(neo4j_create_statemenet)

def execute_transactions(unique_event_nodes_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "password"))
    session = data_base_connection.session()

    for i in unique_event_nodes_execution_commands:
        session.run(i)

execute_transactions(unique_event_nodes_execution_commands)


# Transform df to a list 

In [ ]:
list_all= df.values.tolist()
list_all[4]

### cypher string creation for neo4j 

In [ ]:
# i = similarity_list[6]
# event_node1 = "(e1:Event {event_id: "+ "'" + str(i[1]) + "'" + "})"
# relationship = "[:IS_A]"
# ioc = "(i1:IoC {url: "+ "'" + str(i[0]) + "'" + "})"
# similar_to = "[:SIMILAR_TO {similarity: "+ "'" + str(i[2]) + "'" + "}]"
# ioc2 = "(i2:IoC {url: "+ "'" + str(i[4]) + "'" + "})"
# event_node2 = "(e2:Event {event_id: "+ "'" + str(i[3]) + "'" + "})"
#
# neo4j_create_statemenet = event_node1 + "-" + relationship + "->" + ioc + "-" + similar_to + "->" + ioc2 + "<-" + relationship + "-" + event_node2
# print(neo4j_create_statemenet )

# Create relationship between event node and url

In [ ]:
#change text if needed in merge_left and merge right: 'URL_IS' and Url, url
url_execution_commands = []

for i in list_all:
  match_left = "MATCH (e1:Event {event_id: "+ "'" + str(i[1]) + "'" + "})"
  match_right = "MATCH (e2:Event {event_id: "+ "'" + str(i[3]) + "'" + "})"
  merge_left = "MERGE (e1)-[:URL_IS]->(u1: Url {url: "+ "'" + str(i[0]) + "'" + "})"
  merge_right = "MERGE (e2)-[:URL_IS]->(u2: Url {url: "+ "'" + str(i[4]) + "'" + "})"
  neo4j_statemenet = match_left + " " + match_right + " " + merge_left + " " + merge_right
  url_execution_commands.append(neo4j_statemenet)

def execute_transactions(url_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "password"))
    session = data_base_connection.session()

    for i in url_execution_commands:
        session.run(i)

execute_transactions(url_execution_commands)




# Create relationship (similarity) between two urls from two different events

In [ ]:
#change text if needed in match_left and match_right: 'URL_IS' and Url, url
url_execution_commands = []
similarity_execution_commands = []

for i in list_all:
    match_left = "MATCH (e1:Event {event_id: "+ "'" + str(i[1]) + "'" + "})-[:URL_IS]->(u1: Url {url: "+ "'" + str(i[0]) + "'" + "})"
    match_right = "MATCH (e2:Event {event_id: "+ "'" + str(i[3]) + "'" + "})-[:URL_IS]->(u2: Url {url: "+ "'" + str(i[4]) + "'" + "})"
    merge = "MERGE (u1)-[:IS_SIMILAR_TO {similar: "+ "'" + str(i[2]) + "'" + "}]->(u2)"
    neo4j_statemenet = match_left + " " + match_right + " " + merge
    similarity_execution_commands.append(neo4j_statemenet)

def execute_transactions(similarity_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "password"))
    session = data_base_connection.session()

    for i in similarity_execution_commands:
        session.run(i)

execute_transactions(similarity_execution_commands)

# Monitoring sample nodes 

In [ ]:
mask = df['left_event_id'] == 10
df[mask]

In [ ]:
mask = df['left_event_id'] == 72
df[mask]

In [ ]:
mask = df['left_event_id'] == 96
df[mask]

In [ ]:
mask = df['left_event_id'] == 70
df[mask]